In [ ]:
!pip install crewai['tools'] langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, tool
from langchain_openai import ChatOpenAI
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')

In [ ]:
search_tool = SerperDevTool()

In [ ]:
news_researcher = Agent(
    role = "Senior Researcher",
    goal = "Uncover ground breaking technologies in {topic}",
    backstory=""" You work at a world best tech research company.
    Driven by curiosity, you are at the forefront of innovation, eager to explore and share knowledge that could change the world.""",
    verbose = False,
    allow_delegation = False,
    tools = [search_tool],
    llm = ChatOpenAI(model_name = "gpt-4o",temperature=0.7)
)

news_writer = Agent(
    role = "Expert News Writer",
    goal = "Compelling concise and informative news article about {topic}",
    backstory="""With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner.""" ,
    verbose = True,
    allow_delegation = True,
    llm = ChatOpenAI(model_name = "gpt-4o",temperature=0.7)
)

In [ ]:
task1 = Task(
    description = "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks.",
    expected_output = "A comprehensive 20 paragraphs long report on the latest technology trends. Identify a informative title for each paragraph.",
    agent = news_researcher
)

task2 = Task(
    description = "Compose an insightful article on {topic}."
    "Focus on the latest technology trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive.",
    expected_output = "An informative 16 paragraph article on the latest technology trends of {topic}. Each paragraph should have more than five sentences. All paragraphs should not have the same number of sentences. It should be written a suitable title for each paragraph.",
    agent = news_writer,
    async_execution=False,
    output_file="News_AI_Agent_article.txt"
)

In [ ]:
crew = Crew(
    agents = [news_researcher, news_writer],
    tasks = [task1, task2],
    process = Process.sequential,
    verbose = 1
)

In [ ]:
result=crew.kickoff(inputs={'topic':'AI in Enterprise Resource Planning'})
print(result)

# Agent: Senior Researcher
## Task: Identify the next big trend in AI in Enterprise Resource Planning.Focus on identifying pros and cons and the overall narrative.Your final report should clearly articulate the key points,its market opportunities, and potential risks.


# Agent: Senior Researcher
## Thought: Thought: To identify the next big trend in AI in Enterprise Resource Planning (ERP), I need to gather the latest information on AI developments and their impact on ERP systems. This will help me understand the pros, cons, market opportunities, and potential risks associated with these trends.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"latest trends in AI for Enterprise Resource Planning 2023\"}"
## Tool Output: 

Search results: Title: Enterprise Resource Planning Technology Trends As We Move ...
Link: https://www.forbes.com/councils/forbestechcouncil/2024/11/22/enterprise-resource-planning-technology-trends-as-we-move-toward-2025/
Snippet: Powering In

In [ ]:
# First, ensure necessary libraries are installed
!pip install crewai gradio

import gradio as gr
import os
from crewai import Agent, Task, Crew, Process

# Define Agents
news_researcher = Agent(
    role = "Senior Researcher",
    goal = "Uncover ground breaking technologies in {topic}",
    backstory=""" You work at a world best tech research company.
    Driven by curiosity, you are at the forefront of innovation, eager to explore and share knowledge that could change the world.""",
    verbose = False,
    allow_delegation = False,
    tools = [search_tool],
    llm = ChatOpenAI(model_name = "gpt-4o",temperature=0.7)
)


news_writer = Agent(
    role = "Expert News Writer",
    goal = "Compelling concise and informative news article about {topic}",
    backstory="""With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner.""" ,
    verbose = True,
    allow_delegation = True,
    llm = ChatOpenAI(model_name = "gpt-4o",temperature=0.7)
)

# Function to run the crew and process the output
def run_crew_analysis(topic):
    try:
        # Create tasks dynamically based on the topic
        task1 = Task(
            description = "Identify the next big trend in {topic}."
            "Focus on identifying pros and cons and the overall narrative."
            "Your final report should clearly articulate the key points,"
            "its market opportunities, and potential risks.",
            expected_output = "A comprehensive 20 paragraphs long report on the latest technology trends. Identify a informative title for each paragraph.",
            agent = news_researcher
        )

        task2 = Task(
            description = "Compose an insightful art…nology trends and how it's impacting the industry."
            "This article should be easy to understand, engaging, and positive.",
            expected_output = "An informative 16 paragraph article on the latest technology trends of {topic}. Each paragraph should have more than five sentences. All paragraphs should not have the same number of sentences. It should be written a suitable title for each paragraph.",
            agent = news_writer,
            async_execution=False,

        )

        # Create Crew
        crew = Crew(
            agents=[news_researcher, news_writer],
            tasks=[task1, task2],
            process=Process.sequential,
            verbose=0
        )

        # Kickoff the crew
        result = crew.kickoff(inputs={'topic': topic})

        return result

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Create Gradio Interface with improved handling
def gradio_interface(topic):
    # Use a loading message to indicate processing
    gr.Warning("Processing your request. This may take a few moments...")

    try:
        # Run the analysis
        analysis_result = run_crew_analysis(topic)

        return analysis_result

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Set up Gradio Interface
iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Enter Technology Topic"),
    outputs=gr.Textbox(label="Article"),
    title="CrewAI News Agent",
    description="Enter a technology topic to get a comprehensive analysis"
)

# Launch the interface
iface.launch(debug=True, share=True)
print('Done')

'''# First, ensure necessary libraries are installed
#!pip install crewai gradio

import gradio as gr
from crewai import Agent, Task, Crew, Process

# Define Agents
news_researcher = Agent(
    role = "Senior Researcher",
    goal = "Uncover ground breaking technologies in {topic}",
    backstory=""" You work at a world best tech research company.
    Driven by curiosity, you are at the forefront of innovation, eager to explore and share knowledge that could change the world.""",
    verbose = False,
    allow_delegation = False,
    tools = [search_tool],
    llm = ChatOpenAI(model_name = "gpt-4o",temperature=0.7)
)

news_writer = Agent(
    role = "Expert News Writer",
    goal = "Compelling concise and informative news article about {topic}",
    backstory="""With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner.""" ,
    verbose = True,
    allow_delegation = True,
    llm = ChatOpenAI(model_name = "gpt-4o",temperature=0.7)
)


# Define Tasks
def create_tasks(topic):
    task1 = Task(
    description = "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks.",
    expected_output = "A comprehensive 20 paragraphs long report on the latest technology trends. Identify a informative title for each paragraph.",
    agent = news_researcher
    )

    task2 = Task(
    description = "Compose an insightful art…nology trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive.",
    expected_output = "An informative 16 paragraph article on the latest technology trends of {topic}. Each paragraph should have more than five sentences. All paragraphs should not have the same number of sentences. It should be written a suitable title for each paragraph.",
    agent = news_writer,
    async_execution=False,
    output_file="News_AI_Agent_article.txt"
    )

    return [task1, task2]

# Function to run the crew and process the output
def run_crew_analysis(topic):
    try:
        # Create tasks dynamically based on the topic
        tasks = create_tasks(topic)

        # Create Crew
        crew = Crew(
            agents=[news_researcher, news_writer],
            tasks=tasks,
            process=Process.sequential,
            verbose=1
        )

        # Kickoff the crew
        result = crew.kickoff(inputs={'topic': topic})

        # Read the generated article
        try:
            with open(f'{topic.replace(" ", "_")}_article.txt', 'r') as file:
                article_content = file.read()
        except FileNotFoundError:
            article_content = "Article file not found."

        return result, article_content

    except Exception as e:
        return f"An error occurred: {str(e)}", ""

# Create Gradio Interface
def gradio_interface(topic):
    analysis_result, article_content = run_crew_analysis(topic)

    # Combine results
    full_output = f"Analysis Result:\n{analysis_result}\n\n--- Article Content ---\n{article_content}"

    return full_output

# Set up Gradio
iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Enter Technology Topic"),
    outputs=gr.Textbox(label="Analysis and Article"),
    title="Technology Trend Analysis",
    description="Enter a technology topic to get a comprehensive analysis and article"
)

# Launch the interface
iface.launch(debug=False)'''

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2b4bbbebc5cf6c0db3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Agent: Expert News Writer
## Task: Compose an insightful art…nology trends and how it's impacting the industry.This article should be easy to understand, engaging, and positive.


# Agent: Expert News Writer
## Thought: Thought: To craft a comprehensive and engaging article on the latest technology trends in robotics, I must ensure that I have detailed and up-to-date information about each of the specified trends. I will need to verify and possibly expand on some of these points to create a 16-paragraph article. To begin, I will ask the Senior Researcher to provide more insights or recent data on these specific trends, especially those that may have evolved since the initial context was provided.
## Using tool: Ask question to coworker
## Tool Input: 
"{\"question\": \"Can you provide the latest insights or data on current robotics technology trends, particularly focusing on energy-efficient robots, reshoring, AI integration, collaborative robots (cobots), and their impact on various

'# First, ensure necessary libraries are installed\n#!pip install crewai gradio\n\nimport gradio as gr\nfrom crewai import Agent, Task, Crew, Process\n\n# Define Agents\nnews_researcher = Agent(\n    role = "Senior Researcher",\n    goal = "Uncover ground breaking technologies in {topic}",\n    backstory=""" You work at a world best tech research company.\n    Driven by curiosity, you are at the forefront of innovation, eager to explore and share knowledge that could change the world.""",\n    verbose = False,\n    allow_delegation = False,\n    tools = [search_tool],\n    llm = ChatOpenAI(model_name = "gpt-4o",temperature=0.7)\n)\n\nnews_writer = Agent(\n    role = "Expert News Writer",\n    goal = "Compelling concise and informative news article about {topic}",\n    backstory="""With a flair for simplifying complex topics, you craft"\n    "engaging narratives that captivate and educate, bringing new"\n    "discoveries to light in an accessible manner.""" ,\n    verbose = True,\n    a

In [ ]:
!pwd


/content


In [ ]:
!ls -la .gradio/flagged/


total 44
drwxr-xr-x 2 root root  4096 Dec  8 17:56 .
drwxr-xr-x 3 root root  4096 Dec  8 16:47 ..
-rw-r--r-- 1 root root   131 Dec  8 17:00 dataset1.csv
-rw-r--r-- 1 root root 24095 Dec  8 17:18 dataset2.csv
-rw-r--r-- 1 root root  8116 Dec  8 17:56 dataset3.csv


In [ ]:
from google.colab import files

# Provide the path to the file
file_path = ".gradio/flagged/dataset3.csv"

# Download the file
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
'''# First, ensure necessary libraries are installed
#!pip install crewai gradio

import gradio as gr
import os
from crewai import Agent, Task, Crew, Process

# Define Agents
news_researcher = Agent(
    role = "Senior Researcher",
    goal = "Uncover ground breaking technologies in {topic}",
    backstory=""" You work at a world best tech research company.
    Driven by curiosity, you are at the forefront of innovation, eager to explore and share knowledge that could change the world.""",
    verbose = False,
    allow_delegation = False,
    tools = [search_tool],
    llm = ChatOpenAI(model_name = "gpt-4o",temperature=0.7)
)

news_writer = Agent(
    role = "Expert News Writer",
    goal = "Compelling concise and informative news article about {topic}",
    backstory="""With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner.""" ,
    verbose = True,
    allow_delegation = True,
    llm = ChatOpenAI(model_name = "gpt-4o",temperature=0.7)
)

# Function to run the crew and process the output
def run_crew_analysis(topic):
    try:
        # Create tasks dynamically based on the topic
        task1 = Task(
            description = "Identify the next big trend in {topic}."
            "Focus on identifying pros and cons and the overall narrative."
            "Your final report should clearly articulate the key points,"
            "its market opportunities, and potential risks.",
            expected_output = "A comprehensive 20 paragraphs long report on the latest technology trends. Identify a informative title for each paragraph.",
            agent = news_researcher
        )

        task2 = Task(
            description = "Compose an insightful art…nology trends and how it's impacting the industry."
            "This article should be easy to understand, engaging, and positive.",
            expected_output = "An informative 16 paragraph article on the latest technology trends of {topic}. Each paragraph should have more than five sentences. All paragraphs should not have the same number of sentences. It should be written a suitable title for each paragraph.",
            agent = news_writer,
            async_execution=False,

        )

        # Create Crew
        crew = Crew(
            agents=[news_researcher, news_writer],
            tasks=[task1, task2],
            process=Process.sequential,
            verbose=0
        )

        # Kickoff the crew
        result = crew.kickoff(inputs={'topic': topic})

        return result

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Create Gradio Interface with improved handling
def gradio_interface(topic):
    # Use a loading message to indicate processing
    gr.Warning("Processing your request. This may take a few moments...")

    try:
        # Run the analysis
        analysis_result = run_crew_analysis(topic)

        return analysis_result

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Function to save the result as a text file
def save_result(result):
    # Generate a filename with current timestamp
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"technology_trend_analysis_{timestamp}.txt"

    # Write the result to the file
    with open(filename, 'w') as f:
        f.write(result)

    return filename

# Set up Gradio Interface with save functionality
iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Enter Technology Topic"),
    outputs=[
        gr.Textbox(label="Analysis Result"),
        gr.File(label="Download Analysis")
    ],
    title="Technology Trend Analysis",
    description="Enter a technology topic to get a comprehensive analysis",
    # Modify the flagging options
    flagging_options=["Save"],
    flagging_dir="saved_analyses"
)

# Custom flagging callback to save the file
# Custom flagging callback to save the file
class CustomFlagging(gr.FlaggingCallback):
    def setup(self, interface, **kwargs):
        """Setup method required by FlaggingCallback."""
        pass  # Leave this empty or use it for custom setup logic if needed

    def flag(self, flag_data, flag_option=None, **kwargs):
        # Save the result and return the filename
        result = flag_data[0]  # Assuming the first output is the analysis result
        filename = save_result(result)
        print(f"Saved flagged result to: {filename}")
        return filename

# Set the flagging callback
iface.flagging_callback = CustomFlagging()



# Launch the interface
iface.launch(debug=True, share=True)'''

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://83a10d32aa88359537.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Agent: Expert News Writer
## Task: Compose an insightful art…nology trends and how it's impacting the industry.This article should be easy to understand, engaging, and positive.


# Agent: Expert News Writer
## Using tool: Ask question to coworker
## Tool Input: 
"{\"question\": \"Can you provide more insights and data on the future trends of AI in smartphones, specifically focusing on generative AI, large language models, and multimodal capabilities? I need this information to write an article on AI's impact on the smartphone industry.\", \"context\": \"I am writing an article about the latest technology trends of AI in phones in the future. I have some information about generative AI, large language models (LLMs), and multimodal capabilities, but I need more detailed insights and data to craft an engaging and informative piece.\", \"coworker\": \"Senior Researcher\"}"
## Tool Output: 
Here is the detailed insight into the future trends of AI in smartphones based on the Qualcomm art

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2053, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1809, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1764, in validate_outputs
    raise ValueError(
ValueError: A  function (gradio_interface) didn't return enough output values (needed: 2, returned: 1).
    Output components:
        [textbox, file]
    Output values returned:
        [**Title: Introduction to

# Agent: Expert News Writer
## Task: Compose an insightful art…nology trends and how it's impacting the industry.This article should be easy to understand, engaging, and positive.


# Agent: Expert News Writer
## Thought: I need to gather more detailed and updated information on the latest technology trends of AI in phones in the future to write an engaging and informative article. Specifically, I need to ensure that the content covers various aspects of AI in smartphones, including generative AI, on-device AI, multimodal AI, small language models, customizable AI, edge computing, and sustainability. I will ask the Senior Researcher for insights on these topics to ensure the article is comprehensive and accurate.
## Using tool: Ask question to coworker
## Tool Input: 
"{\"question\": \"What are the latest technology trends and advancements in AI for smartphones expected in the near future, particularly concerning generative AI, on-device AI, multimodal AI, small language models, custom

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2053, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1809, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1764, in validate_outputs
    raise ValueError(
ValueError: A  function (gradio_interface) didn't return enough output values (needed: 2, returned: 1).
    Output components:
        [textbox, file]
    Output values returned:
        [**Title: Introduction to

Created dataset file at: saved_analyses/dataset1.csv


# Agent: Expert News Writer
## Task: Compose an insightful art…nology trends and how it's impacting the industry.This article should be easy to understand, engaging, and positive.


# Agent: Expert News Writer
## Using tool: Ask question to coworker
## Tool Input: 
"{\"question\": \"Could you provide more detailed information and recent statistics on the impact of AI in smartphones, particularly focusing on how generative AI, large language models, and on-device AI processing are shaping the industry? Also, any recent developments in AI-powered gaming and smart home integration would be helpful.\", \"context\": \"The task is to write an informative 16-paragraph article on the latest technology trends of AI in phones in the future. The article should cover various aspects of AI advancements in smartphones, including generative AI, large language models, on-device processing, AI in gaming, and smart home integration. The aim is to make the article engaging, positive, and easy to understa

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2053, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1809, in postprocess_data
    self.validate_outputs(block_fn, predictions)  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1764, in validate_outputs
    raise ValueError(
ValueError: A  function (gradio_interface) didn't return enough output values (needed: 2, returned: 1).
    Output components:
        [textbox, file]
    Output values returned:
        [**1. Introduction: The E

Keyboard interruption in main thread... closing server.


KeyboardInterrupt: 